In [1]:
import cobra.test
import os
from os.path import join
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
from cobra.util import create_stoichiometric_matrix
import numpy as np
model = cobra.io.load_json_model(join("RehMBEL1391_sbml_L3V1.json"))

In [2]:
model

Name,RehMBEL1391
Memory address,0x02780a0a8760
Number of metabolites,1348
Number of reactions,1538
Number of groups,0
Objective expression,1.0*Biomass - 1.0*Biomass_reverse_57a34
Compartments,"c, e"


In [3]:
#figuring out the PHA-production-pathway that is included in the provided model 
for reactions in model.metabolites.get_by_id("phb_c").reactions: 
    print(reactions, reactions.name)  

for reactions in model.metabolites.get_by_id("phb_e").reactions: 
    print(reactions, reactions.name)

PHBt: h_c + phb_c --> h_e + phb_e Poly-beta-hydroxybutyrate transport
PHAS: r3hbcoa_c <=> coa_c + phb_c Polyhydroxyalkanoate synthase
PHBD: h2o_c + phb_c <=> h_c + r3hbn2_c + r3rhbb_c Poly(3-hydroxybutyrate) depolymerase
PHBt: h_c + phb_c --> h_e + phb_e Poly-beta-hydroxybutyrate transport
EX_pbhb_e: phb_e -->  EX_pbhb_e


In [4]:
#figuring out the pathways that lead to r3hbcoa_c to figure out how phb production is connected to the rest of the metabolism
for reactions in model.metabolites.get_by_id("r3hbcoa_c").reactions: 
    print(reactions, reactions.name)

AACOAR: nadp_c + r3hbcoa_c <=> aacoa_c + h_c + nadph_c Acetoacetyl-CoA reductase
3HBCD: r3hbcoa_c <=> b2coa_c + h2o_c 3-hydroxybutyryl-CoA dehydratase
PHAS: r3hbcoa_c <=> coa_c + phb_c Polyhydroxyalkanoate synthase


In [5]:
#looking at the pathway leading to r3hbn2_c
for reactions in model.metabolites.get_by_id("r3hbn2_c").reactions: 
    print(reactions, reactions.name)
#found a dead-end

PHBD: h2o_c + phb_c <=> h_c + r3hbn2_c + r3rhbb_c Poly(3-hydroxybutyrate) depolymerase


In [6]:
#looking at the pathway leading to r3hbb_c
for reactions in model.metabolites.get_by_id("r3rhbb_c").reactions:
    print(reactions)

HBDH: h2o_c + r3rhbb_c <=> 2.0 bhb_c + h_c
PHBD: h2o_c + phb_c <=> h_c + r3hbn2_c + r3rhbb_c


In [7]:
#looking at the pathway leading to b2coa_c
for reactions in model.metabolites.get_by_id("b2coa_c").reactions: 
    print(reactions, reactions.name)

ACOADH1: btcoa_c + fad_c + h_c <=> b2coa_c + fadh2_c Acyl-CoA dehydrogenase butanoyl-CoA)
3HBCD: r3hbcoa_c <=> b2coa_c + h2o_c 3-hydroxybutyryl-CoA dehydratase
ENCOAH1: 3hbcoa_c <=> b2coa_c + h2o_c Enoyl-CoA hydratase
GLUCD1: gcoa_c + nad_c --> b2coa_c + co2_c + nadh_c Glutaryl-CoA dehydrogenase


In [8]:
#looking at the pathway leading to aacoa_c (from r3hbcoa_c reactions)
for reactions in model.metabolites.get_by_id("aacoa_c").reactions: 
    print(reactions, reactions.name)

AACOAR: nadp_c + r3hbcoa_c <=> aacoa_c + h_c + nadph_c Acetoacetyl-CoA reductase
ACACT1r: 2.0 accoa_c <=> aacoa_c + coa_c Acetyl-CoA C-acetyltransferase
3HBCDH: 3hbcoa_c + nadp_c --> aacoa_c + h_c + nadph_c 3-hydroxybutyryl-CoA dehydrogenase
HACD1: 3hbcoa_c + nad_c --> aacoa_c + h_c + nadh_c 3-hydroxyacyl-CoA dehydrogenase (acetoacetyl-CoA)
AACOAS: acac_c + atp_c + coa_c --> aacoa_c + amp_c + h_c + ppi_c Acetoacetyl-CoA synthetase
OCOAT1m: acac_c + succoa_c <=> aacoa_c + succ_c 3-oxoacid CoA-transferase


In [9]:
#to figure out the names of the IDs we printed them
print(model.metabolites.aacoa_c, ' = ', model.metabolites.aacoa_c.name)
print(model.metabolites.acac_c, ' = ', model.metabolites.acac_c.name)
print(model.metabolites.succoa_c, ' = ', model.metabolites.succoa_c.name)
print(model.metabolites.gcoa_c, ' = ',model.metabolites.gcoa_c.name)
print(model.metabolites.oaa_c, ' = ',model.metabolites.oaa_c.name)
print(model.metabolites.accoa_c, ' = ',model.metabolites.accoa_c.name)
print(model.metabolites.btcoa_c, ' = ',model.metabolites.btcoa_c.name)
print(model.metabolites.b2coa_c, ' = ',model.metabolites.b2coa_c.name)
print(model.metabolites.r3rhbb_c, ' = ',model.metabolites.r3rhbb_c.name)
print(model.metabolites.r3hbn2_c, ' = ',model.metabolites.r3hbn2_c.name)
print(model.metabolites.bhb_c, ' = ',model.metabolites.bhb_c.name)

aacoa_c  =  Acetoacetyl-CoA
acac_c  =  Acetoacetate
succoa_c  =  Succinyl-CoA
gcoa_c  =  Glutaryl-CoA
oaa_c  =  Oxaloacetate
accoa_c  =  Acetyl-CoA
btcoa_c  =  Butanoyl-CoA
b2coa_c  =  Crotonoyl-CoA
r3rhbb_c  =  (R)-3-((R)-3-Hydroxybutanoyloxy)butanoate
r3hbn2_c  =  ((R)-3-Hydroxybutanoyl)(n-2)
bhb_c  =  (R)-3-Hydroxybutanoate


In [10]:
#whenever we found a metabolite without an assigned molecular formula we added it in accordance with data found on pubchem
print(model.metabolites.hpimcoa_c.elements)
#add missing molecular formula with data found on PubChem ("https://pubchem.ncbi.nlm.nih.gov/compound/9543022")
model.metabolites.get_by_id("hpimcoa_c").elements = {"C": 28, "H": 46, "N": 7, "O": 20, "P":3, "S":1}
#confirm that the molecular formula was updated
print(model.metabolites.get_by_id("hpimcoa_c").name, 'molecular formula is', model.metabolites.get_by_id("hpimcoa_c").formula) 

#the same was done for other metabolites with missing molecular formulas

print(model.metabolites.r3hbcoa_c.elements)
#add missing molecular formula for r3hbcoa_3
model.metabolites.r3hbcoa_c.elements = {"C": 25,"H": 42,"N": 7,"O": 18,"P": 3,"S": 1}
print('The molecular formula of', model.metabolites.get_by_id("r3hbcoa_c").name, 'is', model.metabolites.get_by_id("r3hbcoa_c").formula)

print(model.metabolites.r3rhbb_c.elements)
print(model.metabolites.r3hbn2_c.elements)
#add molecular formula for r3rhbb_c and r3hbn2_c
model.metabolites.r3rhbb_c.elements = {"C": 8, "H": 13, "O": 5}
model.metabolites.r3hbn2_c.elements = {"C": 4, "H": 6, "O": 2}
#print to check whether it was successful
print('The molecular formula of', model.metabolites.get_by_id("r3rhbb_c").name, 'is', model.metabolites.get_by_id("r3rhbb_c").formula)
print('The molecular formula of', model.metabolites.get_by_id("r3hbn2_c").name, 'is', model.metabolites.get_by_id("r3hbn2_c").formula)

{'C': 27, 'H': 39, 'N': 7, 'O': 20, 'P': 3, 'S': 1}
(3S)-3-Hydroxyadipyl-CoA molecular formula is C27H44N7O20P3S
{}
3-Hydroxypimeloyl-CoA molecular formula is C28H46N7O20P3S
{}
The molecular formula of (R)-3-Hydroxybutanoyl-CoA is C25H42N7O18P3S
{}
{}
The molecular formula of (R)-3-((R)-3-Hydroxybutanoyloxy)butanoate is C8H13O5
The molecular formula of ((R)-3-Hydroxybutanoyl)(n-2) is C4H6O2


Now that we know where and how PHB gets produced in this model, we attempt to introduce our own pathways, according to the enzyme phaC in "real life".

Therefore, pathways for 5 medium-chain-length PHAs were added, namely 
* Polyhydroxyhexanoate (PHHx), 
* Polyhydroxyheptanoate (PHH), 
* Polyhydroxyoctanoate (PHHO), 
* Polyhydroxynonaoate (PHN),
* Polyhydroxydecanoate (PHD).

In [ ]:
#creating metabolites, giving them a unique ID according to the abbreviation of their full name
PHHx = Metabolite(id="phhx_c", compartment ="c", name ="Polyhydroxyhexanoate")
PHH = Metabolite(id="phh_c", compartment = "c", name ="Polyhydroxyheptanoate")
PHHO = Metabolite(id="phho_c", compartment ="c", name ="Polyhydroxyoctanoate")
PHN = Metabolite(id="phn_c", compartment = "c", name ="Polyhydroxynonaoate")
PHD = Metabolite(id="phd_c", compartment = "c", name ="Polyhydroxydecanoate")

PHHx_E = Metabolite(id="phhx_e", compartment ="e", name ="Polyhydroxyhexanoate")
PHH_E = Metabolite(id="phh_e", compartment = "e", name ="Polyhydroxyheptanoate")
PHHO_E = Metabolite(id="phho_e", compartment ="e", name ="Polyhydroxyoctanoate")
PHN_E = Metabolite(id="phn_e", compartment = "e", name ="Polyhydroxynonaoate")
PHD_E = Metabolite(id="phd_e", compartment = "e", name ="Polyhydroxydecanoate")

At first, we find the names of hydroxyacyl-CoA that are already included in the model to later create an reaction where they react to PHA, in case they are fitting.

In [11]:
for metabolite in model.metabolites.query('yl-CoA', 'name'):
    print(metabolite.name, '=', metabolite.id)

Phenylacetyl-CoA = phaccoa_c
2-Methylbut-2-enoyl-CoATiglyl-CoA = 2m2ecoa_c
Anthranilyl-CoA = anthcoa_c
crotonobetainyl-CoA = ctbtcoa_c
Crotonoyl-CoA = b2coa_c
Propanoyl-CoA = ppcoa_c
2-Amino-5-oxocyclohex-1-enecarbonyl-CoA = 2amoeccoa_c
3-Hydroxypropionyl-CoA = 3hpcoa_c
2-Methylprop-2-enoyl-CoA = 2mp2coa_c
O-Succinylbenzoyl-CoA = sbzcoa_c
Pimeloyl-CoA = pmcoa_c
Succinyl-CoA = succoa_c
Glutaryl-CoA = gcoa_c
5-Methyl-3-oxo-4-hexenoyl-CoA = 5m3o4hcoa_c
(2E)-5-Methylhexa-2,4-dienoyl-CoA = 2e5mhdcoa_c
Cyclohexane-1-carboxyl-CoA = cchccoa_c
Malonyl-CoA = malcoa_c
3-Oxopropionyl-CoA = 3oppcoa_c
Adipyl-CoA = adipcoa_c
2-Ketocyclohexane-1-carboxyl-CoA = 2kchccoa_c
Naphthyl-2-methyl-succinyl-CoA = napmsccoa_c
2-Hydroxycyclohexane-1-carboxyl-CoA = hcchccoa_c
3-Methylcrotonyl-CoA = 3mccoa_c
(S)-Methylmalonyl-CoA = mmcoa__S_c
5-Methylhex-4-enoyl-CoA = 5mh4ecoa_c
3-Hydroxypimeloyl-CoA = hpimcoa_c
Naphthyl-2-oxomethyl-succinyl-CoA = napomsuccoa_c
6-Hydroxycyclohex-1-enecarbonyl-CoA = 6hcecoa_c
4-Hydr

Typically, the PHA is composed of monomers that are 2n (n ≥ 0) carbons shorter than the substrates added to the media. For example, growth of _P. oleovorans_ on octanoate ($C\scriptstyle 8$) generates a PHA copolymer containing $C\scriptstyle 8$ and $C\scriptstyle 6$ monomers, whereas growth on dodecanoate ($C\scriptstyle 12$) generates a PHA containing $C\scriptstyle 12$, $C\scriptstyle 10$, $C\scriptstyle 8$, and $C\scriptstyle 6$ monomers (Lageveen et al., 1995). Alkanoic acids present in the media are transported into the cell where they are first converted to CoA esters before being directed to the β-oxidation pathway where a number of 3-hydroxyacyl-CoA intermediates can be generated. 

Since the PHA synthase accepts only the R-isomer of 3-hydroxyacyl- CoA and the bacterial $\beta$-oxidation of saturated fatty acids generates only the S-isomer of 3-hydroxyacyl-CoA, bacteria must have enzymes capable of generating R-3-hydroxyacyl-CoA. One potential enzyme is a 3-hydroxyacyl-CoA epimerase, mediating the reversible conversion of the S- and R-isomers of 3-hydroxyacyl-CoA. 

https://biocyclopedia.com/index/plant_pathways/mediumchainlength_polyhydroxyalkanaote.php

mcl-PHAs have **6-14 carbon atoms** in the side chain, so hydroxy-acids with a 6-14 carbon backbone are required.
 
* (3S)-3-Hydroxyadipyl-CoA has a hexanoic acid backbone (X) -> it can react to polyhydroxyhexanoate.
* 3-Hydroxypimeloyl-CoA has a heptanoic acid backbone (X) -> it can react to polyhydroxyheptanoate.
* 3-Hydroxypropionyl-CoA has a propanoic acid backbone -> not suitable.
* (R)-3-Hydroxybutanoyl-CoA and (S)-3-Hydroxybutanoyl-CoA have a butanoyl backbone -> not suitable.
* (S)-3-Hydroxyisobutyryl-CoA has a methylpropanoic acid backbone -> not suitable.

In the scope of this report, it is assumed that only one type of polyhydroxyalkanoate is produced from the present hydroxyacyls, and the possibility of further degradation of the hydroxyacyl (- 2n carbons) and subsequent incorporation into a polyhydroxyalkanoate is not implemented in the model. Instead, the present hydroxyacyl as well as the added saturated fatty acids are reacting to polyhydroxyalkanoate only in the present version.

As only two suitable hydroxy-acyls were identified, it was decided to add further acids into the medium to enable the simulation of the production of different types of PHA.

In [13]:
model.add_metabolites([Metabolite(id ="hexac_e", name="Hexanoic Acid", compartment = "e", formula = "C6H12O2")])
model.add_metabolites([Metabolite(id ="heptac_e", name="Heptanoic Acid", compartment = "e", formula = "C7H14O2")])
model.add_metabolites([Metabolite(id ="octac_e", name="Octanoic Acid", compartment = "e", formula = "C8H16O2")])
model.add_metabolites([Metabolite(id ="nonac_e", name="Nonanoic Acid", compartment = "e", formula = "C9H18O2")])
model.add_metabolites([Metabolite(id ="decac_e", name="Decanois Acid", compartment = "e", formula = "C10H20O2")])

In [14]:
#creation of R-epimers of (3S)-3-Hydroxyadipyl-CoA
#ID is similar to S-epimer with addition of r
#this time we also know molecular formula (addition of elements as dict does not work, but copy-pasting of formula from model.metabolites.ID-data does)
R3_HADP_COA = Metabolite(id="r3hadpcoa_c", compartment ="c", name ="(3R)-R-Hydroxyadipyl-CoA", formula = 'C27H44N7O20P3S')

epimerase_3hadpcoa_c = Reaction('Epimerisation_of_S-Hydroxyadipyl-CoA')
epimerase_3hadpcoa_c.name = "Epimerisation of S-Hydroxyadipyl-CoA"

#adding stochiometric data and involved metabolites to new reaction to model the epimerisation
epimerase_3hadpcoa_c.add_metabolites({model.metabolites.get_by_id("3hadpcoa_c"): -1,
                                model.metabolites.r3hbcoa_c: 1,})
#adding the reaction to the model
model.add_reactions([epimerase_3hadpcoa_c])

#creating new reaction to model to produce medium-chain-length PHHx from (3R)-R-Hydroxyadipyl-CoA
phaC_PHHx_r3hadpcoa_c = Reaction('phaC_PHHx_r3hadpcoa_c')
phaC_PHHx_r3hadpcoa_c.name = 'mcl-Polyhydroxyalkanoate synthase for PHHx from (3R)-R-Hydroxyadipyl-CoA'

#adding stochiometric data and involved metabolites to new reactions to model the production of PHHx from (3R)-R-Hydroxyadipyl-CoA
phaC_PHHx_r3hadpcoa_c.add_metabolites({R3_HADP_COA: -1,
                                PHHx: 1, 
                                    model.metabolites.coa_c: 1,})

#adding the reaction to the model
model.add_reactions([phaC_PHHx_r3hadpcoa_c])

NameError: name 'PHHx' is not defined